In [1]:
import sys
sys.path.append("../src")
import dotenv
dotenv.load_dotenv()

True

In [2]:
from sitesage_backend import *
prompt = (
    "Open a boutique coffee shop with a cozy vibe targeting young professionals and students. "
    "Strong morning traffic is desired. The location is near 南京东路300号, 黄浦区, 上海."
)

In [3]:
session_id = "0000"
session_dir = ensure_session_dir(session_id)
errors: List[str] = []
assets: Dict[str, Any] = {"reports": {}}

1. understanding

In [10]:
tool_get_map_visualization({"lat":31.235898040745084, "lng":121.47474007623427})


{'provider': 'amap',
 'url': 'https://restapi.amap.com/v3/staticmap?location=121.47474007623427%2C31.235898040745084&zoom=14&size=512%2A512&markers=mid%2C%2CA%3A121.47474007623427%2C31.235898040745084&key=68913ce8db83f7bbe5b4cca9720c7a8f',
 'query_params': {'location': '121.47474007623427,31.235898040745084',
  'zoom': 14,
  'size': '512*512',
  'markers': 'mid,,A:121.47474007623427,31.235898040745084'},
 'origin': {'lat': 31.235898040745084,
  'lng': 121.47474007623427,
  'label': 'origin'},
 'overlays': []}

In [4]:
tool_get_place_info(address = '南京东路300号', city = '上海')

{'provider': 'amap',
 'place_id': 'B00156GMJN',
 'name': '恒基名人购物中心',
 'address': '南京东路300号(南京东路地铁站2号口旁)',
 'category': '购物服务',
 'subcategory': '商场',
 'lat': 31.23787,
 'lng': 121.48409,
 'raw': {'parent': [],
  'distance': [],
  'pcode': '310000',
  'importance': [],
  'biz_ext': {'cost': [], 'rating': '4.7'},
  'recommend': '0',
  'type': '购物服务;商场;购物中心',
  'photos': [{'title': [],
    'url': 'https://aos-comment.amap.com/B00156GMJN/comment/35C10BCB_CCFA_4A5F_B8AD_2F12E1B9B84F_L0_001_1512_201_1748434740115_48809467.jpg'},
   {'title': [],
    'url': 'https://aos-comment.amap.com/B00156GMJN/comment/99AE5023_39E7_43AE_A769_564383F263A6_L0_001_1512_201_1748434740115_75379967.jpg'},
   {'title': [],
    'url': 'https://aos-comment.amap.com/B00156GMJN/comment/83735deaaf684ca4c025759d03d24d19_2048_2048_80.jpg'}],
  'discount_num': '0',
  'gridcode': '4621638812',
  'typecode': '060101',
  'shopinfo': '0',
  'poiweight': [],
  'citycode': '021',
  'adname': '黄浦区',
  'children': [],
  'alias':

In [4]:

# 1) Understanding
understanding_agent = make_understanding_agent()
understanding_prompt = get_understanding_prompt(prompt)
with rt.Session(logging_setting="VERBOSE"):
    resp = await rt.call(understanding_agent, user_input=understanding_prompt)

try:
    ujson = parse_json_from_text(resp.text)
except Exception as e:
    logger.error("Failed to parse understanding agent response as JSON: %s", e)
    ujson = {}
    

[+20.011 s] RT.Session  : DEBUG    - Session 149b947b-88d0-4607-88bd-627bb9f68c42 is initialized
[+20.014 s] RT.Publisher: DEBUG    - RequestCreation(current_node_id=None, new_request_id=1ba08252-ca42-4cdf-b209-b471a802ef75, running_mode=async, new_node_type=EasyToolCallLLM, args=(), kwargs={'user_input': 'Extract store info and resolve the place. Use tools as needed and return the required JSON.\n\nUser request:\nOpen a boutique coffee shop with a cozy vibe targeting young professionals and students. Strong morning traffic is desired. The location is near 南京东路300号, 黄浦区, 上海.'})
[+20.015 s] RT          : INFO     - START CREATED UnderstandingAgent
00:37:04 - LiteLLM:INFO: utils.py:3383 - 
LiteLLM completion() model= gpt-5.1; provider = openai
00:37:06 - LiteLLM:INFO: utils.py:1277 - Wrapper: Completed Call, calling success_handler
[+21.804 s] RT.Publisher: DEBUG    - RequestCreation(current_node_id=445f0e8b-6201-4480-95a7-ea177ff040b4, new_request_id=bf4205f4-632e-462d-a9dc-54684cb93a80

In [6]:
# checkpoint1: save ujson to a file
json.dump(ujson, open("save/understanding.json", 'w'), indent=4, ensure_ascii=False)
with open("save/understanding.md", 'w') as f:
    f.write(ujson["report_md"])

In [5]:
# checkpoint1: load usjon
ujson = json.load(open("save/understanding.json"))

In [6]:

assets["reports"]["01_understanding"] = write_markdown(
    session_dir, "01_understanding", ujson.get("report_md", "# Understanding")
)
store_info = dict(ujson.get("store_info", {}))
place = dict(ujson.get("place", {}))
map_image_url = ujson.get("map_image_url", "")

lat, lng = extract_lat_lng(place)
if lat is not None and lng is not None:
    place["lat"], place["lng"], place["lon"] = lat, lng, lng
    map_image_url = osm_static_map_url(lat, lng)

2. customer

In [7]:

# 2) Customer
customer_agent = make_customer_agent()
customer_prompt = get_customer_prompt(store_info, place)
with rt.Session(logging_setting="VERBOSE"):
    cresp = await rt.call(customer_agent, user_input=customer_prompt)

# Extract the markdown report (the entire response is the report)
customer_report = cresp.text.strip()
assets["reports"]["02_customer"] = write_markdown(
    session_dir, "02_customer", customer_report
)


[+12.634 s] RT.Session  : DEBUG    - Session e9306383-092b-4bc8-9810-0b21a617ec78 is initialized
[+12.636 s] RT.Publisher: DEBUG    - RequestCreation(current_node_id=None, new_request_id=bbd761cd-7c99-48fb-912b-a2c77b5fc76c, running_mode=async, new_node_type=EasyToolCallLLM, args=(), kwargs={'user_input': "Analyze the customer potential for this location.\n\nStore Information:\n{'store_type': 'boutique coffee shop', 'business_description': 'A cozy, boutique-style coffee shop emphasizing high-quality coffee and a comfortable atmosphere suitable for working, studying, and casual meetings.', 'service_mode': 'primarily dine-in with some takeaway; seating designed for staying/working (tables, counters, and small groups).', 'target_customers': ['young professionals', 'university and college students', 'nearby office workers', 'tourists and shoppers in the南京路步行街 area'], 'price_level': 'mid-range (affordable for daily coffee for students and young professionals, not luxury pricing).', 'time_wi

In [8]:
# check point
with open("save/customer.md", 'w') as f:
    f.write(customer_report)

In [14]:
# checkpoint
customer_report = open("save/customer.md").read()

traffic

In [ ]:

# 3) Traffic - receives Customer report
traffic_agent = make_traffic_agent()
traffic_prompt = get_traffic_prompt(store_info, place, customer_report)
with rt.Session(logging_setting="VERBOSE"):
    tresp = await rt.call(traffic_agent, user_input=traffic_prompt)

# Extract the markdown report
traffic_report = tresp.text.strip()
assets["reports"]["03_traffic"] = write_markdown(
    session_dir, "03_traffic", traffic_report
)
